In [1]:
import numpy as np
import pandas as  pd
import random
import requests

# get data directly from REST API and convert it to Pandas Dataframe


In [18]:
# 1st Epoch
url = "https://api.koios.rest/api/v0/pool_delegators?_pool_bech32=pool1vfayajkdc0gw876lax5va6ygpfpsd8kz73k4y977emm3clnxlm5&_epoch_no=328"
r = requests.get(url)
delegator = r.json() # as its a rest api you can directly access the json object 
df1 = pd.DataFrame(delegator)

# 2nd Epoch
url = "https://api.koios.rest/api/v0/pool_delegators?_pool_bech32=pool1vfayajkdc0gw876lax5va6ygpfpsd8kz73k4y977emm3clnxlm5&_epoch_no=329"
r = requests.get(url)
delegator = r.json() # as its a rest api you can directly access the json object 
df2 = pd.DataFrame(delegator)

df = pd.concat([df1, df2])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 0 to 50
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   stake_address  107 non-null    object
 1   amount         107 non-null    object
 2   epoch_no       107 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.3+ KB


In [23]:
df.head(10)

,stake_address,amount,epoch_no,points,point_start,point_end
0,stake1uyd2wl6kqjehqylgcv4q7pp4hhvyhmyxgp07n0t9fvr94fccmvysr,40277485724,328,402,0,0
1,stake1uxmxhnaap40ugkvd87yajnkufgzxca62qt337j2cyejz07q03pdpk,30040095013,328,300,0,0
2,stake1uym7pmcy5xjwrjfjgue6dhkmepvd7l9pmnrm3w6x5dpt5xsslvxfl,20378074572,328,203,0,0
3,stake1uxqudd924qh0hu00gfu3mx5xqmgmc5snvzk5jzu426n59jgmpe92k,20001979904,328,200,0,0
4,stake1uxxee0lq533ejph2n6u3ln9s5pfnvfa7k3htjky4wagwzfsvt2urf,17299175434,328,172,0,0
5,stake1u80mv587rn5hklht8p2yhy0l5lq20y7t5nph4fqpz5ldkqq505j43,15085156587,328,150,0,0
6,stake1uy5cjw7pemtv9l7ezyqraasep329jrw7j93y7kz523egyrglgve7j,15007327791,328,150,0,0
7,stake1u923v8g46qklc5yumwyzelpfw4j520hrl9h5g2qexcfzv9g4x62sl,12847815047,328,128,0,0
8,stake1uxdhjpa5gafwcej24ksr2sxte39qmll6vcynv5fw2kgw9jcftfc5m,11911457718,328,119,0,0
9,stake1u954naz34kzt6uay8vvych05qhdnlwgafpqzj6vmlpnqpacadz0uz,9906728202,328,99,0,0


# Add calculated column
Points is calculated based on 100 ADA. Since the original amount is in Loveflace, you need to divide it by 100M.
1 ADA = 1000000 Loveflace

In [24]:
df[['points']] = df['amount'].astype('int64') / 100000000
df[['points']] = df.points.astype('int64')
df[['point_start']] = 0
df[['point_end']] = 0
df[['stake_address']] = df['stake_address'].astype(str)
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 0 to 50
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   stake_address  107 non-null    object
 1   amount         107 non-null    object
 2   epoch_no       107 non-null    int64 
 3   points         107 non-null    int64 
 4   point_start    107 non-null    int64 
 5   point_end      107 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 5.9+ KB


# Filter out owner address


owner1 = u'stake1uy5cjw7pemtv9l7ezyqraasep329jrw7j93y7kz523egyrglgve7j'
owner2 = 'stake1uxqudd924qh0hu00gfu3mx5xqmgmc5snvzk5jzu426n59jgmpe92k'
owner3 = 'stake1u80mv587rn5hklht8p2yhy0l5lq20y7t5nph4fqpz5ldkqq505j43'
#filter and assign to another df will generate proper index. otherwise if use the same variable as df it will create error 
# in the subsequence logic which use loop
df3 = df[ (df['stake_address'] != owner1) & ( df['stake_address'] != owner2 ) & ( df['stake_address'] != owner2 ) ]
df3.head()
df = df3
df.head()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 0 to 50
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   stake_address  107 non-null    object
 1   amount         107 non-null    object
 2   epoch_no       107 non-null    int64 
 3   points         107 non-null    int64 
 4   point_start    107 non-null    int64 
 5   point_end      107 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 5.9+ KB


In [25]:
end = 0
for i in range(0, len(df)-1):
    df.point_start[i] = end + 1
    df.point_end[i] = df.point_start[i] + df.points[i]
    end = df.point_end[i]

<ipython-input-25-a5c91f0b6454>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.point_start[i] = end + 1
<ipython-input-25-a5c91f0b6454>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.point_end[i] = df.point_start[i] + df.points[i]
<ipython-input-25-a5c91f0b6454>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.point_start[i] = end + 1
<ipython-input-25-a5c91f0b6454>:4: SettingWithCopyWarning: 
A value is trying to be set o

ValueError: No axis named 1 for object type Series

In [26]:
df.head(10)

,stake_address,amount,epoch_no,points,point_start,point_end
0,stake1uyd2wl6kqjehqylgcv4q7pp4hhvyhmyxgp07n0t9fvr94fccmvysr,40277485724,328,402,1,403
1,stake1uxmxhnaap40ugkvd87yajnkufgzxca62qt337j2cyejz07q03pdpk,30040095013,328,300,404,704
2,stake1uym7pmcy5xjwrjfjgue6dhkmepvd7l9pmnrm3w6x5dpt5xsslvxfl,20378074572,328,203,705,908
3,stake1uxqudd924qh0hu00gfu3mx5xqmgmc5snvzk5jzu426n59jgmpe92k,20001979904,328,200,909,1109
4,stake1uxxee0lq533ejph2n6u3ln9s5pfnvfa7k3htjky4wagwzfsvt2urf,17299175434,328,172,1110,1282
5,stake1u80mv587rn5hklht8p2yhy0l5lq20y7t5nph4fqpz5ldkqq505j43,15085156587,328,150,1283,1433
6,stake1uy5cjw7pemtv9l7ezyqraasep329jrw7j93y7kz523egyrglgve7j,15007327791,328,150,1434,1584
7,stake1u923v8g46qklc5yumwyzelpfw4j520hrl9h5g2qexcfzv9g4x62sl,12847815047,328,128,1585,1713
8,stake1uxdhjpa5gafwcej24ksr2sxte39qmll6vcynv5fw2kgw9jcftfc5m,11911457718,328,119,1714,1833
9,stake1u954naz34kzt6uay8vvych05qhdnlwgafpqzj6vmlpnqpacadz0uz,9906728202,328,99,1834,1933


# Get Random Number 

In [29]:

lower_limit = 1
upper_limit = df.point_end.max()
print(upper_limit)
result = random.randint(lower_limit, upper_limit)
print(result)

2682
2224


# Find the Lucky Delegator

In [30]:
pd.options.display.max_colwidth = 100
winner = df[(result >= df['point_start'] ) & (result <= df['point_end'] )]
winner.head()
print("We have a winner!")
print(winner['stake_address'])
print("Points")
print(winner['points'])

We have a winner!
13    stake1u988n2fnagvtc8el0tkc3jzw5e4wh2j0rfwjvcce3lczplgjgskkx
Name: stake_address, dtype: object
Points
13    58
Name: points, dtype: int64
